In [ ]:
# !pip install --upgrade --no-cache-dir biopython
# !pip install rdkit-pypi

In [ ]:
import numpy as np
from collections import Counter
import pandas as pd 
from Bio.Align import substitution_matrices
from sklearn.cluster import KMeans
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import linkage, fcluster
from rdkit import Chem
from rdkit.Chem import AllChem
# from tqdm import tqdm
from tqdm.notebook import tqdm
from itertools import product
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score,roc_auc_score
import gc
import joblib
from joblib import Parallel, delayed
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
import os
import itertools
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import roc_curve, auc
import itertools

from utils import *


In [ ]:
# Hydrophobicity (h1) and Hydrophilicity (h2)
hydrophobicity = {
    'A': 0.62,  'C': 0.29,  'D': -0.90, 'E': -0.74, 'F': 1.19,
    'G': 0.48,  'H': -0.40, 'I': 1.38,  'K': -1.50, 'L': 1.06,
    'M': 0.64,  'N': -0.78, 'P': 0.12,  'Q': -0.85, 'R': -2.53,
    'S': -0.18, 'T': -0.05, 'V': 1.08,  'W': 0.81,  'Y': 0.26
}

hydrophilicity = {
    'A': -0.50, 'C': -1.00, 'D': 3.00,  'E': 3.00,  'F': -2.50,
    'G': 0.00,  'H': -0.50, 'I': -1.80, 'K': 3.00,  'L': -1.80,
    'M': -1.30, 'N': 0.20,  'P': 0.00,  'Q': 0.20,  'R': 3.00,
    'S': 0.30,  'T': -0.40, 'V': -1.50, 'W': -3.40, 'Y': -2.30
}

amino_acids = ['A','C','D', 'E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y']

In [ ]:
secondary_structure = {
"Helix": set("EALMQKRH"),
"Strand": set("VIYCWFT"),
"Coil": set("GNPSD"),
}

In [ ]:
#15
def ctdd(sequence):
    """
    Feature CTDD (Distribution Descriptor of Amino Acid Properties)

    This function calculates the "Distribution" part of the CTD (Composition, 
    Transition, Distribution) descriptor, which describes how amino acids 
    belonging to specific physicochemical property classes are distributed 
    across a protein sequence.

    For each amino acid class (e.g., based on secondary structure propensity),
    it computes five percentile-based positional features:
    - The relative positions (normalized by sequence length) of the first,
      25%, 50%, 75%, and last occurrence of any amino acid in that class.

    Parameters:
    - sequence (str): Protein sequence using one-letter amino acid codes.

    Returns:
    - list: A feature vector of length 15 values
    """
    ctdd_vector = []
    sequence_length=len(sequence)
    for class_name, amino_acids in secondary_structure.items():
        positions = [i for i, aa in enumerate(sequence) if aa in amino_acids]
        
        if not positions:  # If no amino acid of this class is found
            ctdd_vector.extend([0, 0, 0, 0, 0])
            continue

        # Calculate the five key positions (first, 25%, 50%, 75%, last)
        first = positions[0] / sequence_length
        p25 = positions[int(len(positions) * 0.25)] / sequence_length
        p50 = positions[int(len(positions) * 0.50)] / sequence_length
        p75 = positions[int(len(positions) * 0.75)] / sequence_length
        last = positions[-1] / sequence_length

        ctdd_vector.extend([first, p25, p50, p75, last])

    return ctdd_vector

In [ ]:
#Grpups based on their dipoles and side-chain volumes
amino_acid_groups = {
'A': 1, 'G': 1, 'V': 1,  # Group 1
'I': 2, 'L': 2, 'F': 2, 'P': 2,  # Group 2
'Y': 3, 'M': 3, 'T': 3, 'S': 3,  # Group 3
'H': 4, 'N': 4, 'Q': 4, 'W': 4,  # Group 4
'R': 5, 'K': 5,  # Group 5
'D': 6, 'E': 6,  # Group 6
'C': 7   # Group 7
 }

In [ ]:
#343
def ctriad(sequence):
    """
    Feature #343: Conjoint Triad (CTriad)

    This function computes the Conjoint Triad (CTriad) feature vector for a protein sequence.
    It maps amino acids into 7 predefined groups based on dipole moments and side-chain volumes,
    then extracts all overlapping triads (3-residue sliding windows) and counts the frequency
    of each of the 343 possible group combinations (7 × 7 × 7 = 343).

    Components:
    - Reduces each amino acid to one of 7 groups.
    - Forms triads (triplets) using a sliding window across the grouped sequence.
    - Counts and normalizes the frequency of each triad pattern.
    - Produces a 343-dimensional feature vector representing all triad combinations.

    Parameters:
    - sequence (str): Protein sequence using one-letter amino acid codes.

    Returns:
    - np.ndarray: Flattened 343-dimensional vector of normalized triad frequencies.
    """
    # Convert sequence to reduced alphabet (group numbers)
    reduced_seq = [amino_acid_groups[aa] - 1 for aa in sequence if aa in amino_acid_groups]
    # Extract triads
    triads = [tuple(reduced_seq[i:i+3]) for i in range(len(reduced_seq) - 2)]
   
    # Count occurrences of each triad
    triad_counts = Counter(triads)

    # Normalize counts
    total_triads = len(triads)
    triad_vector = np.zeros((7, 7, 7))  # 7^3 possible triads

    for triad, count in triad_counts.items():
        triad_vector[triad] = count / total_triads  # Normalize frequency

    return triad_vector.flatten()

In [ ]:
#400
def dde(sequence):
    """
    Feature #400: DDE (Dipeptide Deviation from Expected Mean)

    This function calculates the DDE (Dipeptide Deviation from Expected mean) descriptor
    for a given protein sequence. It evaluates how the observed frequency of each possible
    dipeptide deviates from its expected frequency under the assumption of amino acid independence.

    Components:
    - Dc: Observed frequency of each of the 400 dipeptides (20 × 20).
    - Tm: Theoretical mean frequency of each dipeptide (product of individual amino acid frequencies).
    - Tv: Theoretical variance assuming independent occurrence.
    - DDE: Z-score-like value capturing standardized deviation (Dc - Tm) / sqrt(Tv)

    This descriptor is useful in capturing **non-random pairwise amino acid associations** 
    and provides insight into sequence-specific dipeptide usage biases.

    Parameters:
    - sequence (str): Protein sequence using one-letter amino acid codes.

    Returns:
    - list: A 400-dimensional feature vector representing the DDE of all possible dipeptides.
    """
    
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'  
    dipeptides = [''.join(pair) for pair in itertools.product(amino_acids, repeat=2)]
    
    aa_counts = Counter(sequence)
    L = len(sequence)
    aa_freq = {aa: aa_counts.get(aa, 0) / L for aa in amino_acids}

    dipeptide_counts = Counter([sequence[i:i+2] for i in range(L-1)])
    Dc = {dp: dipeptide_counts.get(dp, 0) / (L-1) for dp in dipeptides}

    Tm = {dp: aa_freq[dp[0]] * aa_freq[dp[1]] for dp in dipeptides}
    Tv = {dp: (Tm[dp] * (1 - Tm[dp])) / L if Tm[dp] > 0 else 0 for dp in dipeptides}

    DDE = {dp: (Dc[dp] - Tm[dp]) / (Tv[dp] ** 0.5) if Tv[dp] > 0 else 0 for dp in dipeptides}

    return list(DDE.values())

In [ ]:
#5
hydrophobicity_Kyte_Doolittle  = {
    'A': 1.8,  'C': 2.5,  'D': -3.5, 'E': -3.5, 'F': 2.8,
    'G': -0.4, 'H': -3.2, 'I': 4.5,  'K': -3.9, 'L': 3.8,
    'M': 1.9,  'N': -3.5, 'P': -1.6, 'Q': -3.5, 'R': -4.5,
    'S': -0.8, 'T': -0.7, 'V': 4.2,  'W': -0.9, 'Y': -1.3
}




def geary_autocorrelation(sequence, max_lag=5, property_dict=hydrophobicity_Kyte_Doolittle):
    """
    Feature #5: Geary Autocorrelation Descriptor

    This function calculates the Geary autocorrelation descriptor for a given protein sequence,
    which quantifies the spatial autocorrelation of a chosen physicochemical property across 
    different lags (distances) in the amino acid sequence.

    Concept:
    - For a selected amino acid property (e.g., hydrophobicity, polarity), this feature measures 
      how similar the property values are between amino acids separated by a lag `d`.
    - It is based on the Geary's C metric (used in spatial statistics), which is sensitive to 
      local changes in the property values across the sequence.

    Formula:
    Geary(d) = (N - 1) * Σ[(Pᵢ - Pᵢ₊ₗ)²] / [2 * (N - d) * Σ(Pᵢ - mean(P))²]

    Parameters:
    - sequence (str): Protein sequence using one-letter amino acid codes.
    - max_lag (int): Maximum distance (lag) to consider for autocorrelation (default = 5).
    - property_dict (dict): Dictionary mapping amino acids to a numeric property scale,
                            such as hydrophobicity, hydrophilicity, polarity, or molecular weight.

    Returns:
    - list: A list of `max_lag` Geary autocorrelation values for lags from 1 to `max_lag`.
    """
    prop_values = np.array([property_dict.get(aa, 0) for aa in sequence])  # Default 0 if AA is unknown
    N = len(prop_values)
    mean_p = np.mean(prop_values)

    geary_values = {}
    
    for d in range(1, max_lag + 1):
        numerator = np.sum((prop_values[:-d] - prop_values[d:]) ** 2)
        denominator = 2 * (N - d) * np.sum((prop_values - mean_p) ** 2)
        geary_values[f'Geary_Lag_{d}'] = (N - 1) * numerator / denominator if denominator != 0 else 0

    return list(geary_values.values())

In [ ]:
df_train=read_data_from_file(file_name_ncbi_datas)

df_val=read_data_from_file(gasaid_test_file_name)

df_test_gasaid=read_data_from_file(gasaid_test_file_name)
df_test_ncbi=read_data_from_file(ncbi_test_file_name)

df_test = pd.concat([df_test_gasaid, df_test_ncbi], ignore_index=True)

In [ ]:
df_Asian_flu=read_data_from_file(Asian_flu_test)
df_hong_kong_flu=read_data_from_file(hong_kong_flu_test)
df_pdmh1n1_flu=read_data_from_file(pdmh1n1_flu_test)
df_covid= read_data_from_file(covid_test)
df_cows=read_data_from_file(cows_test)

In [ ]:
print(len(df_train))
print(len(df_val))

In [ ]:
df_train = remove_duplicates(df_train)
df_val = remove_duplicates(df_val)

print(len(df_train))
print(len(df_val))

In [ ]:
df_train=remove_common_sequences(df_train,df_test)
df_train=remove_common_sequences(df_train,df_val)
df_train=remove_common_sequences(df_train,df_Asian_flu)
df_train=remove_common_sequences(df_train,df_hong_kong_flu)
df_train=remove_common_sequences(df_train,df_covid)
df_train=remove_common_sequences(df_train,df_cows)



print(len(df_train))
print(len(df_val))

In [ ]:
# ---- Define sizes of each feature set ----
vec_size = 30 + 15 + 343 + 400 + 5   

In [ ]:
def ExtractFeatures(S):
    V = np.zeros((len(S), vec_size))
    
    for i, seq in enumerate(tqdm(S, desc="Processing sequences")):
        offset = 0

        # APAAC: 30 features
        V[i, offset:offset + 30] = apaac(seq)
        offset += 30

        # CTDD: 15 features
        V[i, offset:offset + 15] = ctdd(seq)
        offset += 15

        # CTriad: 343 features
        V[i, offset:offset + 343] = ctriad(seq)
        offset += 343

        # DDE: 400 features
        V[i, offset:offset + 400] = dde(seq)
        offset += 400

        # Geary Autocorrelation: 5 features
        V[i, offset:offset + 5] = geary_autocorrelation(seq)
        offset += 5

    return V

In [ ]:
def getXY(df):
    X = ExtractFeatures(df['Sequence'])
    print(f"X.shape is {X.shape}")
    Y = np.array((df["Class"].str.lower() != "human").astype(int))
    print(f"y_train.size is {Y.size}")
    return X,Y

In [ ]:
X_train,y_train = getXY(df_train)

In [ ]:
X_val,y_val = getXY(df_val)

In [ ]:
plot_tsne(X_val,y_val)

In [ ]:
rf = RandomForestClassifier(n_estimators=100,random_state=42)
rf.fit(X_train, y_train)

In [ ]:
preds = RandomForest(X_val,y_val, rf)
con_matrix(y_val,preds)

In [ ]:
joblib.dump(rf, 'RF_ML_Features.pkl')

In [ ]:
lr = LogisticRegression(max_iter=1000, random_state=42)
lr.fit(X_train, y_train)

In [ ]:
preds = LogisticReg(X_val,y_val, lr)
con_matrix(y_val,preds)

In [ ]:
# Save
joblib.dump(lr, 'LR_ML_Features.pkl')

In [ ]:
svm = SVC(kernel='rbf',random_state=42,probability=True)
svm.fit(X_train, y_train)

In [ ]:
preds = SVM(X_val,y_val, svm)
con_matrix(y_val,preds)

In [ ]:
joblib.dump(svm, 'SVM_ML_Features.pkl')